<a href="https://colab.research.google.com/github/superchargez/nlp_tasks/blob/main/baichuan_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install auto-gptq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 111.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 M

In [4]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import argparse

model_name_or_path = "TheBloke/baichuan-vicuna-7B-GPTQ"
model_basename = "baichuan-vicuna-7b-GPTQ-4bit-128g.no-act.order"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=False,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)

# Note: check the prompt template is correct for this model.
prompt = "Tell me about AI"
prompt_template=f'''USER: {prompt}
ASSISTANT:'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.1, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

# Prevent printing spurious transformers error when using pipeline with AutoGPTQ
logging.set_verbosity(logging.CRITICAL)

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

print(pipe(prompt_template)[0]['generated_text'])




*** Generate:
USER: Tell me about AI
ASSISTANT: Artificial intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. AI systems can process vast amounts of data, identify patterns, and make decisions based on that information.

There are several types of AI, including:

1. Narrow or weak AI: These systems are designed to perform a specific task, such as playing chess or recognizing objects in images.
2. General or strong AI: These systems are designed to perform a wide range of tasks, similar to humans.
3. Superintelligent AI: These systems are theoretically capable of surpassing human intelligence and achieving superhuman intelligence.

AI systems can be trained using various techniques, including supervised learning, unsupervised learning, and reinforcement learning. The accuracy and effectiveness of AI systems can vary depending on the type of task, the quality of the data used to train them, and the algorithms

In [13]:
prompt = "Write python code to reverse string"
prompt_template=f'''You are helpful AI assistant and your job is to follow instructions by user faithfully and if you encounter code related question then you will only write the code and will not try to generate or estimate its output also avoid examples and explanation{prompt}
ASSISTANT:'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.1, max_new_tokens=2048)
print(tokenizer.decode(output[0]))



*** Generate:
You are helpful AI assistant and your job is to follow instructions by user faithfully and if you encounter code related question then you will only write the code and will not try to generate or estimate its output also avoid examples and explanationWrite python code to reverse string
ASSISTANT: Sure, here's a Python code snippet to reverse a string:
```python
def reverse_string(s):
    return s[::-1]

# Example usage
s = "Hello, world!"
print(reverse_string(s))  # Output: "olleH,dlroW!"
```
This code snippet defines a function called `reverse_string` that takes a string as input and returns its reverse. The string is first reversed using slicing with `s[::-1]`, and then the result is printed to the console.</s>
